# Talk-openpyxl
## Reading and writing Microsoft Excel Files with openpyxl

![](images/AutomateTheBoringStuff.png)

https://automatetheboringstuff.com/

https://nostarch.com/automate-boring-stuff-python-3rd-edition

* Introducing one of the handiest Python books
* 2nd edition free to read online
* 1st part is learn to program, 2nd part shows how to use Python to automate common tasks
* Most of what I've learned about openpyxl is from the "Working with Excel Spreadsheets"
* 3rd edition adds chapters on: Google Sheets, SQLite Databases, Making graphs, Recognizing Text in images, Text to speech and Speech Recognition Engines

## 1 - Open an Excel Workbook

In [1]:
from openpyxl import load_workbook

wb = load_workbook('FoodInspectionsFirst100.xlsx')
wb

In [2]:
# Attempt to load something that isn't an Excel workbook
# catch the exception

from sys import stderr
from openpyxl.utils.exceptions import InvalidFileException

try:
    load_workbook('FoodInspectionsFirst100.csv')
except InvalidFileException as e:
    print(e, file=stderr)

openpyxl does not support .csv file format, please check you can open it with Excel first. Supported formats are: .xlsx,.xlsm,.xltx,.xltm


In [3]:
# This can also raise FileNotFound and other standard I/O errors
try:
    load_workbook('notthere.xlsx')
except FileNotFoundError as e:
    print(e, file=stderr)

[Errno 2] No such file or directory: 'notthere.xlsx'


## 2 - Create a workbook view, select, add and remove Worksheets

In [4]:
from openpyxl import Workbook

wb = Workbook()
wb

In [5]:
# By default workbook has 1 worksheet
wb.sheetnames

['Sheet']

In [6]:
# Get the active sheet (note this also a writeable property)
sheet = wb.active
sheet

<Worksheet "Sheet">

In [7]:
# You can think of a Workbook as dict of Worksheets
sheet = wb['Sheet']
sheet

<Worksheet "Sheet">

In [8]:
# Change the title of the sheet and check sheetnames
sheet.title = "README"
wb.sheetnames

['README']

In [9]:
# Add a sheet to the workbook
failed_sheet = wb.create_sheet('Failed')
wb.sheetnames

['README', 'Failed']

In [10]:
# insert a sheet at specified index
passed_sheet = wb.create_sheet('Passed', index=1)
wb.sheetnames

['README', 'Passed', 'Failed']

In [11]:
# Make Passed the active worksheet
wb.active = passed_sheet
wb.active

<Worksheet "Passed">

In [12]:
# add another Worksheet so we can move it then delete it
delete_me = wb.create_sheet('DELETEME')
wb.sheetnames

['README', 'Passed', 'Failed', 'DELETEME']

In [13]:
wb.move_sheet(delete_me, offset=-1)
wb.sheetnames

['README', 'Passed', 'DELETEME', 'Failed']

In [14]:
# Now remove the Worksheet
wb.remove(delete_me)
wb.sheetnames

['README', 'Passed', 'Failed']

In [15]:
# You'll get a ValueError if the sheet isn't present
try:
    wb.remove(delete_me)
except ValueError:
    print(f"{delete_me.title} not present might as well delete the variable", file=stderr)
    del delete_me

DELETEME not present might as well delete the variable


## 3 - Read data from a worksheet

In [16]:
# Load the spreadsheet again
inspections_wb = load_workbook('FoodInspectionsFirst100.xlsx')
inspections_wb

In [17]:
inspections_wb.sheetnames

['FoodInspectionsFirst100']

In [18]:
del sheet
in_sheet = inspections_wb.active
in_sheet

<Worksheet "FoodInspectionsFirst100">

In [19]:
# boundaries of the sheet max_row and max_column
print(f"{in_sheet.max_row=}")
print(f"{in_sheet.max_column=}")

in_sheet.max_row=101
in_sheet.max_column=17


### SPREADSHEET RULES: Row and column indexes start with 1 not 0!

In [20]:
# Iterate over the row of headings
row = 1
for col in range(1, in_sheet.max_column + 1):
    cell = in_sheet.cell(row, col)
    print(f"{row=} {col=} value={cell.value}")

row=1 col=1 value=Inspection ID
row=1 col=2 value=DBA Name
row=1 col=3 value=AKA Name
row=1 col=4 value=License #
row=1 col=5 value=Facility Type
row=1 col=6 value=Risk
row=1 col=7 value=Address
row=1 col=8 value=City
row=1 col=9 value=State
row=1 col=10 value=Zip
row=1 col=11 value=Inspection Date
row=1 col=12 value=Inspection Type
row=1 col=13 value=Results
row=1 col=14 value=Violations
row=1 col=15 value=Latitude
row=1 col=16 value=Longitude
row=1 col=17 value=Location


In [21]:
# you can select a cell using a string
from string import ascii_uppercase

heading_map = {}
for col in ascii_uppercase[:17]:
    cell = in_sheet[f"{col}1"]
    heading_map[cell.value] = col
heading_map

{'Inspection ID': 'A',
 'DBA Name': 'B',
 'AKA Name': 'C',
 'License #': 'D',
 'Facility Type': 'E',
 'Risk': 'F',
 'Address': 'G',
 'City': 'H',
 'State': 'I',
 'Zip': 'J',
 'Inspection Date': 'K',
 'Inspection Type': 'L',
 'Results': 'M',
 'Violations': 'N',
 'Latitude': 'O',
 'Longitude': 'P',
 'Location': 'Q'}

In [22]:
unique_result_values = set()
for i in range(2, in_sheet.max_row):
    key = f"M{i}"
    cell = in_sheet[key]
    unique_result_values.add(cell.value)
unique_result_values

{'Fail', 'Pass', 'Pass w/ Conditions'}

In [23]:
headings = list(heading_map.keys())
max_heading_len = max(len(h) for h in headings)
print(f"{max_heading_len=}")
record = {h: in_sheet[f"{heading_map[h]}2"].value for h in heading_map.keys()}
record

max_heading_len=15


{'Inspection ID': 1068208,
 'DBA Name': 'CHINA COURT RESTAURANT',
 'AKA Name': 'CHINA COURT RESTAURANT',
 'License #': 2141795,
 'Facility Type': 'Restaurant',
 'Risk': 'Risk 1 (High)',
 'Address': '1146 N MILWAUKEE AVE',
 'City': 'CHICAGO',
 'State': 'IL',
 'Zip': '60642',
 'Inspection Date': datetime.datetime(2012, 3, 14, 0, 0),
 'Inspection Type': 'License Re-Inspection',
 'Results': 'Fail',
 'Violations': '18. NO EVIDENCE OF RODENT OR INSECT OUTER OPENINGS PROTECTED/RODENT PROOFED, A WRITTEN LOG SHALL BE MAINTAINED AVAILABLE TO THE INSPECTORS - Comments: FOUND NO PEST CONTROL LOG BOOK ON PREMISES,FRONT/REAR DOORS TO PREMISES WITH 1/4" MODE OF ENTRY.MUST PROVIDE PEST CONTROL LOG BOOK,RODENT PROOF SAID DOORS ALL ACROSS THE BOTTOMS. | 32. FOOD AND NON-FOOD CONTACT SURFACES PROPERLY DESIGNED, CONSTRUCTED AND MAINTAINED - Comments: SPLASHGUARD AT FRONT EXPOSED HANDSINK IS MISSING MUST PROVIDE,SINK STOPPERS ARE NEEDED FOR THE 3 COMPARTMENT SINK.   | 41. PREMISES MAINTAINED FREE OF LITTER

## 4 - Converting between int and str column indexes

In [24]:
from openpyxl.utils import (
    column_index_from_string,
    get_column_letter,
)
int_col = 27
print(f"{column_index_from_string('AA')=}")
print(f"{get_column_letter(27)=}")

column_index_from_string('AA')=27
get_column_letter(27)='AA'


## 5 - Write data to Spreadsheet and Save Workbook

In [25]:
# Create a new workbook
output_wb = Workbook()
output_wb

readme_sheet = output_wb.active
readme_sheet.title = 'README'

pass_sheet = output_wb.create_sheet('PASSED')
fail_sheet = output_wb.create_sheet('FAILED')



In [26]:
from itertools import count

pass_headings = [
    'Inspection ID','DBA Name', 'License #',
    'Address', 'Inspection Date', 'Results',
]

fail_headings = pass_headings.copy()
fail_headings.append('Violations')

all_headings = fail_headings.copy()

def write_headings(sheet, headings):
    for col, heading in enumerate(headings, 1):
        cell = sheet.cell(1, col)
        cell.value = heading

write_headings(pass_sheet, pass_headings)
write_headings(fail_sheet, fail_headings)

In [27]:
pass_row_count, fail_row_count = count(1), count(1)
for in_row in range(2, in_sheet.max_row + 1):    
    record = {}
    for heading in all_headings:
        col_str = heading_map[heading]
        record[heading] = in_sheet[f"{col_str}{in_row}"].value
    print(record)
    # for heading, side in HEADINGS_OF_INTEREST.items():
    #     in_col = heading_map[heading]
    #     key = f"{in_col}{in_row}"
    #     record[heading] = in_sheet[key].value
    # print(record)
    # results = record['Results']
    # pass_col_count, fail_col_count = count(1), count(1)
    break
    


            
            

{'Inspection ID': 1068208, 'DBA Name': 'CHINA COURT RESTAURANT', 'License #': 2141795, 'Address': '1146 N MILWAUKEE AVE', 'Inspection Date': datetime.datetime(2012, 3, 14, 0, 0), 'Results': 'Fail', 'Violations': '18. NO EVIDENCE OF RODENT OR INSECT OUTER OPENINGS PROTECTED/RODENT PROOFED, A WRITTEN LOG SHALL BE MAINTAINED AVAILABLE TO THE INSPECTORS - Comments: FOUND NO PEST CONTROL LOG BOOK ON PREMISES,FRONT/REAR DOORS TO PREMISES WITH 1/4" MODE OF ENTRY.MUST PROVIDE PEST CONTROL LOG BOOK,RODENT PROOF SAID DOORS ALL ACROSS THE BOTTOMS. | 32. FOOD AND NON-FOOD CONTACT SURFACES PROPERLY DESIGNED, CONSTRUCTED AND MAINTAINED - Comments: SPLASHGUARD AT FRONT EXPOSED HANDSINK IS MISSING MUST PROVIDE,SINK STOPPERS ARE NEEDED FOR THE 3 COMPARTMENT SINK.   | 41. PREMISES MAINTAINED FREE OF LITTER, UNNECESSARY ARTICLES, CLEANING  EQUIPMENT PROPERLY STORED - Comments: DETAIL CLEAN,ORGANIZE REAR STORAGE AREAS AND REMOVE ALL CLUTTER/UNUSED KITCHEN EQUIPMENTS DUE TO POTENTIAL PEST HARBORAGE.'}


In [28]:
output_wb.save('output.xlsx')

## 6 - Additional items to fill time.

1. You can select ranges i.e. A1:D20
2. Inserting rows into a workheet
3. If you insert a row into a table, you need to also alter the bounds of the table
4. I ran into an issue inserting into a table which was the source of the pivot table

## 7 - Resources

* https://github.com/jocassid/Talk-openpyxl
* https://openpyxl.readthedocs.io